# Capstone Project Final : Best location for opening an Indian restaurant in Manhatan




## Table of contents
* [1 - Introduction: Business Problem](#introduction)
* [2 - Data Description to solve the problem](#data)
* [3 - Methodology](#methodology)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)

## 1 - Introduction: Business Problem <a name="introduction"></a>

The goal of the project is to open a Indian Restaurant in Manhatan, so the business problem, is to find the best place for opening the restaurant.
The target will be a investor who want to opening a restaurant, and to help him to find the best place base on Data Science.

The main obejective is to find the best neighborhood for opening based on these criterias :
* Attractivity of the area
* no competition (no other Lebanese restaurant nearby)
* great potential ( People use to or like eating lebanese food)
* Area with lot of restaurant


The fist step will be to find the top neighborhood where you can find lebanese restaurant.
The second will be to rate them, to find the best neighborhood.
And finally to find a similar neighborhood with no lebanese restaurant, for opening e new one.




## 2 - Data Description to solve the problem <a name="data"></a>

For this Business problem, we will use the API FourSquare for :

* Get and analyse the venues of Paris
* Finding the neighborhoods, where you can find Lebanese restaurant
* Analyse each fo these neighborhood to find the best one, based on these criterias : Good revenues, Attractive area, good reviews etc...
* Find a similar neighborhood, but with no lebanese restaurant opened yet.


    
    

### Restaurant analysis in Paris

We will import the useful livbraries and use FourQuare API for the analysis

In [16]:
# Import Libraries

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

from bs4 import BeautifulSoup #Library for web scrapping
import urllib.request 

import json # library to handle JSON files


from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans



#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
#!pip install folium
#import folium # map rendering library

print('Libraries imported.')

Libraries imported.


#### Downloading the data set of New York Neighborhood

In [17]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

wget: error while loading shared libraries: libcrypto.so.1.0.0: cannot open shared object file: No such file or directory
Data downloaded!


In [18]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)


FileNotFoundError: [Errno 2] No such file or directory: 'newyork_data.json'

In [19]:
neighborhoods_data = newyork_data['features']
neighborhoods_data[0]

NameError: name 'newyork_data' is not defined

#### Transform the data into a pandas dataframe

In [20]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

loop through the data and fill the dataframe one row at a time.

In [21]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

NameError: name 'neighborhoods_data' is not defined

In [24]:
neighborhoods.head()

ModuleNotFoundError: No module named 'pandas.io.formats.html'

  Borough Neighborhood   Latitude  Longitude
0   Bronx    Wakefield  40.894705 -73.847201
1   Bronx   Co-op City  40.874294 -73.829939
2   Bronx  Eastchester  40.887556 -73.827806
3   Bronx    Fieldston  40.895437 -73.905643
4   Bronx    Riverdale  40.890834 -73.912585

Display the shape of the data set

We will focus only on MAnhatan sector

In [22]:
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude


In [23]:
print('The dataframe of Manahatan has {} neighborhoods.'.format(manhattan_data.shape[0]
    )
)

The dataframe of Manahatan has 0 neighborhoods.


Geographical coordinates of Manhattan : 

In [24]:
address = 'Manhattan, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Manhattan are 40.7900869, -73.9598295.


Now we will vizualize Manhatan and all it's neigborhood 

In [25]:
# create map of Manhattan using latitude and longitude values
map_manhattan = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)  
    
map_manhattan

NameError: name 'folium' is not defined

The next step is to use Foursquare API to explore and analyse the neigborhood, and to find where are the lebanese restaurant

#### Foursquare Credentials and Version definition

In [ ]:

CLIENT_ID = 'UQORBT4J0DMW2MWBCIFHZHJM3KJA1G0ZZOBAHNXVTNDOYWYO' # your Foursquare ID
CLIENT_SECRET = '5VBPSSBVFQFFMILW50QUEK2GHIBD05ZIFNA1YUW10TOK01N5' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)
LIMIT = 100 

## 3 - Methodology <a name="methodology"></a>

-------------------- TO UPDATE !!!!!*********************-------------------------------------/
In this project we will direct our efforts on detecting areas of Berlin that have low restaurant density, particularly those with low number of Italian restaurants. We will limit our analysis to area ~6km around city center.

In first step we have collected the required data: location and type (category) of every restaurant within 6km from Berlin center (Alexanderplatz). We have also identified Italian restaurants (according to Foursquare categorization).

Second step in our analysis will be calculation and exploration of 'restaurant density' across different areas of Berlin - we will use heatmaps to identify a few promising areas close to center with low number of restaurants in general (and no Italian restaurants in vicinity) and focus our attention on those areas.

In third and final step we will focus on most promising areas and within those create clusters of locations that meet some basic requirements established in discussion with stakeholders: we will take into consideration locations with no more than two restaurants in radius of 250 meters, and we want locations without Italian restaurants in radius of 400 meters. We will present map of all such locations but also create clusters (using k-means clustering) of those locations to identify general zones / neighborhoods / addresses which should be a starting point for final 'street level' exploration and search for optimal venue location by stakeholders.

#### Using Foursquare we will serch for lebanese Restaurant by neighborhood


In [ ]:
# type your answer here
##<!-- The correct answer is:
LIMIT = 1000 # limit of number of venues returned by Foursquare API
##-->

##<!--
radius = 500 # define radius

#### Define a function tothe venies  in Manhattan¶

In [ ]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
## Get the venues for Manhatan area :

manhattan_venues = getNearbyVenues(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude']
                                  )


In [ ]:
LIMIT

In [ ]:

#Display the information
print(manhattan_venues.shape)
manhattan_venues.head()

Here's are the number of venues returned for each neighborhood: 

In [ ]:
manhattan_venues.groupby('Neighborhood').count()

Now we will verify if we have Indian Restaurant In Manhatan adn display the NEighborhood:

In [ ]:
df_Indian_rest = manhattan_venues[manhattan_venues['Venue Category'].str.contains('Indian Restaurant')]
df_Indian_rest

Get the the number of Unique Venues

In [ ]:
print('There are {} uniques categories.'.format(len(manhattan_venues['Venue Category'].unique())))

### Data Preparation for clustering the Neighborhood of Manhatan

In [ ]:
# we will trnaform the data frame in order to have the Venue Category in column
manhattan_onehot = pd.get_dummies(manhattan_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = manhattan_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot.head()

In [ ]:
manhattan_onehot.shape

#### Now we will group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [ ]:
manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()
manhattan_grouped

In [ ]:
manhattan_grouped.shape

In [ ]:
## select the neigborhood with lebanese restaurant
df_Indian_rest = manhattan_grouped[manhattan_grouped['Indian Restaurant'] > 0]
df_Indian_rest

##### Let's view for each neighborhood the top 10 Venues 

In [ ]:
#Function  to sort the venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

DataFrame creation : the top 10 venues for each neighborhood.

In [ ]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

### Cluster Neighborhoods


We will use k-means to cluster the neighborhood into 5 clusters.


In [ ]:
# set number of clusters
kclusters = 5

manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

Creation of a new data frame to have for each Neighborhood : The Cluster, and the 10 top Venues 

In [ ]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

manhattan_merged = manhattan_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

manhattan_merged.head() # check the last columns!

#### Now let's see for each Neighborhood the number of Indian Restaurant

In [ ]:
df_GroupByVenues =  manhattan_venues.loc[:,['Neighborhood','Neighborhood Latitude','Neighborhood Longitude','Venue Category']]
#ufo.loc[:, 'City']
#df_GroupByVenues = manhattan_venues.loc[:,['Borough','Neighborhood','Latitude','Longitude']]
#df.loc[1:3, ['Name', 'Qualification']] 
##df_GroupByVenues = manhattan_venues.groupby('Venue Category').groups
#df_GroupByVenues

#manhattan_onehot.groupby('Neighborhood').mean().reset_index()
df_GroupByVenues_Count= df_GroupByVenues.groupby(['Neighborhood','Neighborhood Latitude','Neighborhood Longitude'])['Venue Category'].count().reset_index()

#data.groupby(['month', 'item'])['date'].count()
df_GroupByVenues_Count


#### Define Foursquare Credentials and Version


#### Explore Neighborhoods in Paris
Let's create a function to repeat the same process to all the neighborhoods in Paris

In [ ]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)